In [4]:
%%capture
from transformers import MarianMTModel, MarianTokenizer
import torch

# Helsinki-NLP/opus-mt-en-zh
# Helsinki-NLP/opus-mt-zh-en
# junyanz/marianmt-zh-classical_en

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
zh_to_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

In [18]:
embeddings = zh_to_en.get_input_embeddings()
input_ids = tokenizer("我想买一辆车", return_tensors="pt").input_ids

# pass input_ids to encoder
encoder_hidden_states = zh_to_en.base_model.encoder(input_ids, return_dict=True).last_hidden_state

# change the input and pass to encoder
input_ids_perturbed = tokenizer("我想买一套房子",return_tensors="pt").input_ids
encoder_hidden_states_perturbed = zh_to_en.base_model.encoder(input_ids_perturbed,
                                                              return_dict=True).last_hidden_state

# compare shape and encoding of first vector
print(f"length of input embeddings {embeddings(input_ids).shape[1]}, \nlength of encoder_hidden_states {encoder_hidden_states.shape[1]}")

# compare values of word embedding of "我" for input_ids and perturbed input_ids
print("Is encoding for '我' equal to its perturbed version?:", torch.allclose(encoder_hidden_states[0,0], encoder_hidden_states_perturbed[0,0], atol=1e-3))

# print("Is encoding for `I` equal to its perturbed version?: ", torch.allclose(encoder_hidden_states[0, 0], encoder_hidden_states_perturbed[0, 0], atol=1e-3))

length of input embeddings 5, 
length of encoder_hidden_states 5
Is encoding for '我' equal to its perturbed version?: False


# The annotated transformer (transformer model from scratch)

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline